In [6]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

link = "https://www.holidify.com/explore/"
link_start = "https://www.holidify.com/explore"
p_html = requests.get(link).text
soup = BeautifulSoup(p_html, 'lxml')

containers = soup.find_all('div', class_ = "col-12 col-md-6 pr-lg-3 pl-md-3 pl-lg-0 result")

col = ["Place", "State", "Rating(/5)", "Details", "Landmarks", "Starting Prices", "Link"]
df = pd.DataFrame(columns=col)

for container in containers:

    place = container.find('h3', class_ = "card-heading").text.strip().split(',')

    lnk = container.find('div', class_ = "card content-card").a['href']
    lnk = link_start + lnk

    city_num = place[0]
    city = city_num.split(' ')[1]

    state = city
    if len(place)>1:
        state = place[1].strip()
    
    rating = container.find('span', class_ = "rating-badge").text.strip().split()
    rating = rating[0]

    about = container.find('p', class_ = "card-text").text.strip()

    landmarks = container.find('p', class_ = "mb-3").text.strip().split('\n')
    lm = []
    for land in landmarks:
        land = land.strip()
        if ':' not in land:
            if land != "":
                lm.append(land)
    
    price = container.find('p', class_ = "collection-cta")
    if price == None:
        price = "NaN"
    else:
        price = price.text.strip().split()[-1]
    
    df.loc[len(df.index)] = ([city, state, rating, about, lm, price, lnk])
    
df.to_csv('HolidayDestinations.csv', index = False)
